# Lab: Metaclass

Write a metaclass that allows us to use `class` syntax to create a `dict`, e.g.

```python
class MyDict(metaclass=Dictify):
    a = 1
    b = 2
```

creates the dict `{'a': 1, 'b': 2}`

Hint: Metaclasses and class decorators are not required to return classes!

In [1]:
class Dictify(type):
    def __new__(meta, name, bases, dct):
        return {
            k: v for k, v in dct.items()
            if not k.startswith('_')
        }
    
class MyDict(metaclass=Dictify):
    a = 1
    b = 2
    c = 'foo'
    
MyDict

{'a': 1, 'b': 2, 'c': 'foo'}

In [2]:
def dictify(name, bases, dct):
    return {
        k: v for k, v in dct.items()
        if not k.startswith('_')
    }

class MyDict2(metaclass=dictify):
    a = 1
    b = 2
    c = 'foo'
    
MyDict2

{'a': 1, 'b': 2, 'c': 'foo'}

# Lab: Descriptor

Write a (non-data) descriptor that will allow us to calculate an attribute value the *first* time it is loaded, and cache it for subsequent loads.

In [21]:
class reify:
    def __init__(self, getter):
        self._getter = getter
        
    def __get__(self, instance, type):
        print('Calling the descriptor')
        if instance is None:
            return self
        value = self._getter(instance)
        instance.__dict__[self._getter.__name__] = value
        return value

class MyClass:
    
    @reify
    def a(self):
        print('Calculate a!')
        return 'a'

In [22]:
obj = MyClass2()
obj.a

Calling the descriptor
Calculate a!


'a'

In [23]:
obj.a

'a'

Implement the descriptor above as a *data* descriptor:

In [32]:
sentry = object()

class reify():
    def __init__(self, getter):
        self._getter = getter
        
    def __get__(self, instance, typ):
        #print('Calling the descriptor')
        if instance is None:
            return self
        cache = getattr(instance, '_instance_cache', {})
        instance._instance_cache = cache
        cached_value = cache.get(id(self), sentry)
        if cached_value is sentry:
            cached_value = self._getter(instance)
            cache[id(self)] = cached_value
        return cached_value

class MyClassData():
    
    @reify
    def a(self):
        print('Calculate a!')
        return 'a'

In [33]:
obj = MyClassData()
obj.a

Calculate a!


'a'

In [34]:
obj.a

'a'

In [35]:
obj1 = MyClassData()
obj1.a

Calculate a!


'a'

In [36]:
obj2 = MyClass()
obj2.a

Calling the descriptor
Calculate a!


'a'

In [37]:
%timeit obj1.a

581 ns ± 13.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [38]:
%timeit obj2.a

50.5 ns ± 0.788 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
